In [1]:
import pandas as pd
import numpy as np
import re
#import matplotlib.pyplot as plt
import datetime

In [2]:
ratings_rb = pd.read_csv('ratings_rb.csv', keep_default_na=False)

In [3]:
beer_rb = pd.read_csv('./RateBeer/beers.csv')


In [4]:
brew_rb = pd.read_csv('./RateBeer/breweries.csv')

In [5]:
users = pd.read_csv('users.csv')

In [6]:
breweries = pd.read_csv('brew.csv')

### Length of review

In [7]:
# Regex to replace all non alphabet/space characters with a space (This version tries to keep diacritic and accented characters)
# This is not a perfect 100% encompassing solution. Users sometimes have typos or other irregularities in their text e.g. 'bottlePour' being counted as 1 word.
# Phrases such as 0.33ml might also be separated into two mistakenly such as '0' and '33ml'
def remove_punct(review):
    return re.sub('[^\w\s]', ' ', review)
    
def count_words(review):
    return len(review.split())

In [8]:
rb_raw_text = ratings_rb['text'].apply(remove_punct)

In [9]:
review_len_rb = rb_raw_text.apply(count_words)

In [10]:
ratings_rb['text_length'] = review_len_rb

### Relative Time of review

In [13]:
first_review_time_rb = ratings_rb.groupby('beer_name')['date'].min()
first_review_time_rb

beer_name
#5 Belgian Golden Tripel        1500199200
#Freedom The Berry Amendment    1382871600
#Freedom Unalienable Rights     1382871600
#Poekhali Spas IPA              1485601200
#Poekhali!! Belka               1468231200
                                   ...    
Żywiec Szampańskie              1475488800
Žemaičių Alus                   1215511200
’t Goeye Goet Dubbel            1222164000
北京赤 红云生 Buzz                    1494842400
爱啤                              1493805600
Name: date, Length: 396448, dtype: int64

In [14]:
def relative_time_rb(time, beer_name):
    return time - first_review_time_rb.loc[beer_name]

In [15]:
ratings_rb['relative_time'] = ratings_rb.apply(lambda x: relative_time_rb(x['date'], x['beer_name']), axis=1)

### Time of writing review

In [16]:
ratings_rb['review_time'] = ratings_rb['date'].apply(datetime.datetime.fromtimestamp)

### Country of beer and country of user

In [17]:
user_df = users[['user_id', 'location']].set_index(keys='user_id')

In [24]:
ratings_rb['user_id'] = ratings_rb['user_id'].apply(str)

In [23]:
users['user_id']

0                nmann08.184925
1            stjamesgate.163714
2                 mdagnew.19527
3         helloloser12345.10867
4               cypressbob.3708
                  ...          
223873                   134893
223874                   327816
223875                   183139
223876                    82020
223877                    48123
Name: user_id, Length: 223878, dtype: object

In [26]:
ratings_rb = ratings_rb.merge(how='left' ,right=user_df, on='user_id')

In [27]:
ratings_rb

,beer_name,beer_id,date,user_name,user_id,overall,text,text_length,relative_time,review_time,location
0,33 Export (Gabon),410549,1461664800,Manslow,175852,8,"Puszka 0,33l dzięki Christoph . Kolor jasnozło...",18,0,2016-04-26 12:00:00,Poland
1,Castel Beer (Gabon),105273,1487329200,MAGICuenca91,442761,8,Cerveza pale lager gabonesa. MÃ¡s floja que la...,66,242010000,2017-02-17 12:00:00,Spain
2,Castel Beer (Gabon),105273,1466762400,Sibarh,288889,5,"Kolor- złoty, klarowny. Piana - drobna, średni...",23,221443200,2016-06-24 12:00:00,Poland
3,Castel Beer (Gabon),105273,1451646000,fombe89,250510,5,"Botella, de GabÃ³n regalo familiar.31/01/2015C...",16,206326800,2016-01-01 12:00:00,Spain
4,Castel Beer (Gabon),105273,1445594400,kevnic2008,122778,7,Many thanks for this beer to Erzengel. Pours l...,22,200275200,2015-10-23 12:00:00,Germany
...,...,...,...,...,...,...,...,...,...,...,...
7123801,Svejk Blonde,220897,1411034400,Travlr,83882,10,"Draft at the source. Clear golden color, fluff...",26,25138800,2014-09-18 12:00:00,"United States, North Carolina"
7123802,Svejk Blonde,220897,1385895600,TBone,10233,10,"Tap @brewpub, TiraneClear golden color, good c...",31,0,2013-12-01 12:00:00,Finland
7123803,Svejk Dark,220898,1415098800,Rob_D_UK,257161,9,In their beer garden after a walking tour arou...,27,29203200,2014-11-04 12:00:00,Spain
7123804,Svejk Dark,220898,1410861600,Travlr,83882,6,"Draft at the source. Hazy maroon color, tan he...",24,24966000,2014-09-16 12:00:00,"United States, North Carolina"


In [28]:
beer_to_brew_rb = beer_rb[['beer_id', 'brewery_id']].set_index(keys='beer_id')['brewery_id']
beer_to_brew_rb = beer_to_brew_rb.apply(str)

In [29]:
brew_locations = breweries[['id', 'location']].set_index(keys='id')['location']

In [30]:
brew_rb['id'] = brew_rb['id'].apply(str)
brew_rb = brew_rb[['id', 'location']].set_index(keys='id')['location']
def beer_country_rb(identifier):
    brewery = beer_to_brew_rb.loc[identifier]
    return brew_rb.loc[brewery]

In [31]:
ratings_rb['beer_location'] = ratings_rb['beer_id'].apply(beer_country_rb)

In [32]:
ratings_rb

,beer_name,beer_id,date,user_name,user_id,overall,text,text_length,relative_time,review_time,location,beer_location
0,33 Export (Gabon),410549,1461664800,Manslow,175852,8,"Puszka 0,33l dzięki Christoph . Kolor jasnozło...",18,0,2016-04-26 12:00:00,Poland,Gabon
1,Castel Beer (Gabon),105273,1487329200,MAGICuenca91,442761,8,Cerveza pale lager gabonesa. MÃ¡s floja que la...,66,242010000,2017-02-17 12:00:00,Spain,Gabon
2,Castel Beer (Gabon),105273,1466762400,Sibarh,288889,5,"Kolor- złoty, klarowny. Piana - drobna, średni...",23,221443200,2016-06-24 12:00:00,Poland,Gabon
3,Castel Beer (Gabon),105273,1451646000,fombe89,250510,5,"Botella, de GabÃ³n regalo familiar.31/01/2015C...",16,206326800,2016-01-01 12:00:00,Spain,Gabon
4,Castel Beer (Gabon),105273,1445594400,kevnic2008,122778,7,Many thanks for this beer to Erzengel. Pours l...,22,200275200,2015-10-23 12:00:00,Germany,Gabon
...,...,...,...,...,...,...,...,...,...,...,...,...
7123801,Svejk Blonde,220897,1411034400,Travlr,83882,10,"Draft at the source. Clear golden color, fluff...",26,25138800,2014-09-18 12:00:00,"United States, North Carolina",Albania
7123802,Svejk Blonde,220897,1385895600,TBone,10233,10,"Tap @brewpub, TiraneClear golden color, good c...",31,0,2013-12-01 12:00:00,Finland,Albania
7123803,Svejk Dark,220898,1415098800,Rob_D_UK,257161,9,In their beer garden after a walking tour arou...,27,29203200,2014-11-04 12:00:00,Spain,Albania
7123804,Svejk Dark,220898,1410861600,Travlr,83882,6,"Draft at the source. Hazy maroon color, tan he...",24,24966000,2014-09-16 12:00:00,"United States, North Carolina",Albania
